In [8]:
import torch
from scipy.io import wavfile
import torch.utils.data as Data
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [13]:
sample_rate_original,waveform_original,_= wavfile.read('data\\2_sets\\A\\'+'A_1_min.wav')
_,waveform_1,_  = wavfile.read('data\\2_sets\\A\\'+'A_d1.wav')
_,waveform_2,_ = wavfile.read('data\\2_sets\\A\\'+'A_d2.wav')
_,waveform_3,_ = wavfile.read('data\\2_sets\\A\\'+'A_d3.wav')
_,waveform_4,_ = wavfile.read('data\\2_sets\\A\\'+'A_n1.wav')
_,waveform_5,_ = wavfile.read('data\\2_sets\\A\\'+'A_n2.wav')
_,test_audio,_ = wavfile.read('data\\2_sets\\B\\'+'B_d1.wav') 
waveform_original = waveform_original.astype('float32')
waveform_original /= 255
print("Shape of waveform: {}".format(np.shape(waveform_original)))
print("Sample rate of waveform: {}".format(sample_rate_original))
print(np.shape(waveform_original))



Shape of waveform: (3068033, 2)
Sample rate of waveform: 44100
(3068033, 2)


In [14]:
samples=100
BATCH_SIZE = 20
waveform_1 = waveform_1.astype('float32')
waveform_1 /= 255
waveform_2 = waveform_2.astype('float32')
waveform_2 /= 255
waveform_3 = waveform_3.astype('float32')
waveform_3 /= 255
waveform_4 = waveform_4.astype('float32')
waveform_4 /= 255
waveform_5 = waveform_5.astype('float32')
waveform_5 /= 255
test_audio = test_audio.astype('float32')
test_audio /=255

waveform_original = np.repeat(torch.tensor(np.transpose(waveform_original))[np.newaxis,...], BATCH_SIZE, axis=0)
y1 = np.repeat(torch.tensor(np.transpose(waveform_1))[np.newaxis,...], samples, axis=0)
y2 = np.repeat(torch.tensor(np.transpose(waveform_2))[np.newaxis,...], samples, axis=0)
y3 = np.repeat(torch.tensor(np.transpose(waveform_3))[np.newaxis,...], samples, axis=0)
y4 = np.repeat(torch.tensor(np.transpose(waveform_4))[np.newaxis,...], samples, axis=0)
y5 = np.repeat(torch.tensor(np.transpose(waveform_5))[np.newaxis,...], samples, axis=0)

y = np.concatenate((y1,y2,y3,y4,y5),axis=0)
print(np.shape(waveform_original))


np.random.shuffle(y)



print(np.shape(y3))
print(y3)

torch.Size([20, 2, 3068033])
torch.Size([100, 2, 3068033])
tensor([[[0.0667, 0.0431, 0.0510,  ..., 0.9490, 0.8549, 0.7765],
         [0.0706, 0.0392, 0.0431,  ..., 0.9608, 0.8510, 0.7686]],

        [[0.0667, 0.0431, 0.0510,  ..., 0.9490, 0.8549, 0.7765],
         [0.0706, 0.0392, 0.0431,  ..., 0.9608, 0.8510, 0.7686]],

        [[0.0667, 0.0431, 0.0510,  ..., 0.9490, 0.8549, 0.7765],
         [0.0706, 0.0392, 0.0431,  ..., 0.9608, 0.8510, 0.7686]],

        ...,

        [[0.0667, 0.0431, 0.0510,  ..., 0.9490, 0.8549, 0.7765],
         [0.0706, 0.0392, 0.0431,  ..., 0.9608, 0.8510, 0.7686]],

        [[0.0667, 0.0431, 0.0510,  ..., 0.9490, 0.8549, 0.7765],
         [0.0706, 0.0392, 0.0431,  ..., 0.9608, 0.8510, 0.7686]],

        [[0.0667, 0.0431, 0.0510,  ..., 0.9490, 0.8549, 0.7765],
         [0.0706, 0.0392, 0.0431,  ..., 0.9608, 0.8510, 0.7686]]])


In [17]:
torch.manual_seed(1)

  # size of the minibatch
#z = torch.from_numpy(y)
# load dataset into dataLoader
#print(type(z))
loader = Data.DataLoader(

    dataset=y,

    batch_size=BATCH_SIZE,

    shuffle=True,

    num_workers=0,

)



class DiscreteNet(torch.nn.Module):
        # use an embedding layer to build a NN

    def __init__(self):

        super(DiscreteNet, self).__init__()

        self.embedding = nn.LSTM(3068033,10)
        self.embedding2 = nn.Linear(10,3068033)


    def forward(self, x):

        y,x = self.embedding(x)
        y = self.embedding2(y)
        return y



f = DiscreteNet()


optimizer = optim.Adam(params=f.parameters(), lr=0.001)
loss = nn.MSELoss()
sami

def run():
    for epoch in range(100):

        for step,(batch_x) in enumerate(loader):
            optimizer.zero_grad()
            print(np.shape(batch_x))
            fx = f(batch_x)
            print(np.shape(fx),np.shape(waveform_original))
            l = loss(fx,waveform_original)
            l.backward()
            optimizer.step()
            print("Epoch: ",epoch,"Loss: ",l.item())

    print('Finished Training')


run()

test_audio = torch.tensor(test_audio)
d = f(test_audio.reshape(1,2,3068033))
d = d.reshape(2, 3068033)
d*=255
wavfile.write('reconstructed_B_blank.wav',sample_rate_original,np.transpose(d.detach().numpy()))



torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  0 Loss:  60.824607849121094
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  0 Loss:  60.80070495605469
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  0 Loss:  60.76587677001953
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  0 Loss:  60.7373161315918
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  0 Loss:  60.70461654663086
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  0 Loss:  60.6562614440918
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  0 Loss:  60.63917922973633
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  0 Loss:  60.59810256958008
torch.Size([20, 2, 306803

torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  2 Loss:  58.547454833984375
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  2 Loss:  58.75081253051758
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  2 Loss:  58.47718048095703
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  2 Loss:  58.59242630004883
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  2 Loss:  58.608882904052734
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  2 Loss:  58.27466583251953
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  2 Loss:  58.422996520996094
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  2 Loss:  58.5659065246582
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 306

Epoch:  5 Loss:  56.595096588134766
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  5 Loss:  56.77106857299805
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  5 Loss:  56.59919357299805
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  5 Loss:  56.29135513305664
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  5 Loss:  57.577144622802734
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  5 Loss:  56.53852844238281
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  5 Loss:  56.98066329956055
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  5 Loss:  55.993751525878906
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  5 Loss:  56.3

torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  8 Loss:  54.680686950683594
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  8 Loss:  55.839195251464844
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  8 Loss:  55.16950607299805
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  8 Loss:  54.703041076660156
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  8 Loss:  55.01329803466797
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  8 Loss:  56.08795928955078
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  8 Loss:  54.66168212890625
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  8 Loss:  55.21664810180664
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 30

torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  10 Loss:  52.83047103881836
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  10 Loss:  54.13071823120117
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  10 Loss:  54.18474578857422
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  10 Loss:  53.73535919189453
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  10 Loss:  52.980186462402344
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  10 Loss:  52.322025299072266
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  11 Loss:  53.16621017456055
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  11 Loss:  52.47726058959961
torch.Size([20, 2, 3068033])
torch.Size([20

...

torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  88 Loss:  26.672914505004883
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  88 Loss:  23.630741119384766
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  88 Loss:  21.5368595123291
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  88 Loss:  25.099477767944336
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  88 Loss:  25.879709243774414
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  88 Loss:  26.289199829101562
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  88 Loss:  23.217304229736328
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  88 Loss:  23.058263778686523
torch.Size([20, 2, 3068033])
torch.Size

Epoch:  90 Loss:  24.877429962158203
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  90 Loss:  22.442119598388672
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  90 Loss:  22.628049850463867
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  90 Loss:  25.991504669189453
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  90 Loss:  22.990163803100586
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  90 Loss:  22.42638397216797
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  90 Loss:  23.2420711517334
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  90 Loss:  21.212743759155273
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  91 

torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  93 Loss:  23.789411544799805
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  93 Loss:  20.40349769592285
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  93 Loss:  22.33404541015625
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  93 Loss:  23.806900024414062
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  93 Loss:  21.735815048217773
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  93 Loss:  22.023462295532227
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  93 Loss:  23.26466178894043
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  93 Loss:  20.597448348999023
torch.Size([20, 2, 3068033])
torch.Size(

Epoch:  96 Loss:  19.38722801208496
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  96 Loss:  20.504301071166992
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  96 Loss:  25.499757766723633
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  96 Loss:  20.219722747802734
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  96 Loss:  22.255796432495117
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  96 Loss:  22.320720672607422
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  96 Loss:  19.91322135925293
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  96 Loss:  19.300813674926758
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  96

torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  98 Loss:  21.486284255981445
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  98 Loss:  22.521528244018555
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  98 Loss:  28.02961540222168
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  98 Loss:  19.87254524230957
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  98 Loss:  19.386632919311523
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  98 Loss:  20.464834213256836
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  98 Loss:  21.85990333557129
torch.Size([20, 2, 3068033])
torch.Size([20, 2, 3068033]) torch.Size([20, 2, 3068033])
Epoch:  98 Loss:  19.985708236694336
torch.Size([20, 2, 3068033])
torch.Size(

In [20]:
_,test_audio_1,_ = wavfile.read('data\\2_sets\\B\\'+'B_n1.wav') 
test_audio_1 = test_audio_1.astype('float32')
test_audio_1 /=255
test_audio_1 = torch.tensor(test_audio_1)
n = f(test_audio_1.reshape(1,2,3068033))
n = n.reshape(2, 3068033)


wavfile.write('reconstructed_B_noisy.wav',sample_rate_original,np.transpose(n.detach().numpy()))



In [1]:
from sklearn.metrics import mean_squared_error 

sample_rate_original,waveform_original_B,_= wavfile.read('data\\2_sets\\B\\'+'B_1_min.wav')
waveform_original_B = waveform_original_B.astype('float32')
waveform_original_B /= 255
waveform_original_B = np.repeat(torch.tensor(np.transpose(waveform_original_B))[np.newaxis,...], BATCH_SIZE, axis=0)
# Calculation of Mean Squared Error (MSE) 
mean_squared_error(waveform_original_B[0].detach().numpy(),d.detach().numpy()) 


NameError: name 'wavfile' is not defined